In [ ]:
! pip install datasets transformers rouge-score nltk
! apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login
from google.colab import drive
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Log in to huggingface with access tokens
notebook_login()

In [ ]:
# mount drive to get wikihow datasets
# Put wikihowAll.csv under Mydrive (If not change data_dir accordingly)

# drive.mount('/content/drive')
raw_datasets = load_dataset("cnn_dailymail", "3.0.0")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model_checkpoint = "t5-small"
trained_path = "bubblecookie/t5-small-finetuned-cnndm-samsum"
tokenizer = AutoTokenizer.from_pretrained(trained_path)
metric = load_metric("rouge")
prefix = "summarize: "

In [ ]:
# preprocess the dataset by tokenizer

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
# Load trained model from hugging face (Change name for each model)
model_trained = AutoModelForSeq2SeqLM.from_pretrained(trained_path)

In [ ]:
# args
model_name = model_checkpoint.split("/")[-1]
test_file_name = f"{model_name}-t5-small-finetuned-samsum-cnn"
args = Seq2SeqTrainingArguments(
    test_file_name,
    predict_with_generate=True,
    fp16=True
)

# data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_trained)

# metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model = model_trained,
    args = args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [ ]:
predictions = trainer.predict(tokenized_datasets["test"], max_length = max_target_length)

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, highlights, article. If id, highlights, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 11490
  Batch size = 8


In [ ]:
print(predictions.metrics)

{'test_loss': 1.6586518287658691, 'test_rouge1': 40.7973, 'test_rouge2': 18.6692, 'test_rougeL': 29.0199, 'test_rougeLsum': 38.024, 'test_gen_len': 73.603, 'test_runtime': 2059.9532, 'test_samples_per_second': 5.578, 'test_steps_per_second': 0.698}


In [ ]:
# Checking examples

import pandas as pd
from IPython.display import display, HTML
import numpy as np

def show_results(raw_datasets, predictions, num_example = 3, random = False):
  text = []
  headline = []
  prediction = []
  num_data = len(raw_datasets)
  if random:
    picks = np.random.choice(num_data, size = num_example, replace = False)
  else:
    picks = np.arange(num_example)
  for i in picks:
    text += [raw_datasets["article"][i]]
    headline += [raw_datasets["highlights"][i]]
    decoded = tokenizer.decode(predictions.predictions[i])
    decoded = decoded.replace('<pad>', '')
    decoded = decoded.replace('</s>', '')
    prediction += [decoded]

  df = {"text": text, "headline": headline, "prediction": prediction}
  df = pd.DataFrame(df)
  display(HTML(df.to_html()))

show_results(raw_datasets["test"], predictions, num_example = 20)